In [4]:
import sys
import os
import datetime
import time
import pandas as pd
import numpy as np
import re
pd.options.display.latex.repr = True
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib import rcParams
plt.rcParams['font.sans-serif'] = ['STFangsong']
plt.rcParams['font.size'] = 5
plt.rc('figure', figsize=(6, 4))
sys.path.append('/Users/yantingting/PycharmProjects/modeling/')
import database_conncet as data_con
import data_processing as dp
import data_encryption as de
from data_io_utils import *
import plot_tools as pt
import warnings
warnings.filterwarnings('ignore')

In [5]:
file_path = '/Users/yantingting/Seafile/风控/模型/10 印尼/202003 催收模型/'
if not os.path.exists(file_path):
    os.mkdir(file_path)

# 取数

In [11]:
query0  = '''
select t1.*,
t2.newusermodelscorev5,
t3.oldusermodelv1score,
t4.newUserModelScoreV6,
t5.oldUserModelV2Score
from 
(select id::text  as loan_id,return_flag,effective_date,due_date,loan_no
from rt_t_gocash_core_loan
where loan_status not like '%PAIDOFF%' and due_date>='2020-04-05') t1 
left join
(select distinct loanid,
case when length(newusermodelscorev5)>10 then newusermodelscorev5::json->>'newUserModelScoreV5' else newusermodelscorev5 end as newusermodelscorev5
from risk_mongo_gocash_installmentriskcontrolparams
where newusermodelscorev5 <>'') t2 
on t1.loan_id = t2.loanid 
left join 
(select loanid,oldusermodelv1score
from rt_risk_mongo_gocash_installmentriskcontrolresult
where oldusermodelv1score is not null) t3 
on t1.loan_id = t3.loanid 
left join 
(select orderno as loanid,cast(outputmap::json->'newUserModelScoreV6' AS varchar) as newUserModelScoreV6
from rt_risk_mongo_gocash_riskreport 
where ruleresultname = 'newUserModelScoreV6') t4 
on t1.loan_id = t4.loanid
left join 
(select orderno as loanid,cast(outputmap::json->'oldUserModelV2Score' AS varchar) as oldUserModelV2Score
from rt_risk_mongo_gocash_riskreport 
where ruleresultname = 'oldUserModelV2Score') t5
on t1.loan_id = t5.loanid;
'''

In [8]:
query_temp = '''
select t1.*,
t2.newusermodelscorev5,
t3.oldusermodelv1score,
t4.newUserModelScoreV6,
t5.oldUserModelV2Score
from 
(select t1.id as case_id,
order_id::varchar as loan_id,
t1.customer_id,
t1.product,
date(t1.due_date) as due_date,
t1.late_day,
t2.return_flag,
t3.refund_type
from dw_gocash_gocash_collection_col_case t1 
left join rt_t_gocash_core_loan t2 
on t1.order_id = t2.id
left join dw_gocash_gocash_collection_col_repayment t3 
on t1.id = t3.case_id
where date(t1.due_date)>='2020-04-05' and date(t1.due_date)<'2020-04-10' 
and t1.order_status !='PAIDOFF' 
and (t1.product = 'UKU' or t1.product ='CASHCASH')) t1 
left join
(select distinct loanid,
case when length(newusermodelscorev5)>10 then newusermodelscorev5::json->>'newUserModelScoreV5' else newusermodelscorev5 end as newusermodelscorev5
from risk_mongo_gocash_installmentriskcontrolparams
where newusermodelscorev5 <>'') t2 
on t1.loan_id = t2.loanid 
left join 
(select loanid,oldusermodelv1score
from rt_risk_mongo_gocash_installmentriskcontrolresult
where oldusermodelv1score is not null) t3 
on t1.loan_id = t3.loanid 
left join 
(select orderno as loanid,cast(outputmap::json->'newUserModelScoreV6' AS varchar) as newUserModelScoreV6
from rt_risk_mongo_gocash_riskreport 
where ruleresultname = 'newUserModelScoreV6') t4 
on t1.loan_id = t4.loanid
left join 
(select orderno as loanid,cast(outputmap::json->'oldUserModelV2Score' AS varchar) as oldUserModelV2Score
from rt_risk_mongo_gocash_riskreport 
where ruleresultname = 'oldUserModelV2Score') t5
on t1.loan_id = t5.loanid;

'''

In [9]:
df0 = data_con.database_gn(query_temp)
save_data_to_pickle(df0,file_path,'data_0414.pkl')
df0.shape
df0.head()

Successful connection


(11047, 12)

,case_id,loan_id,customer_id,product,due_date,late_day,return_flag,refund_type,newusermodelscorev5,oldusermodelv1score,newusermodelscorev6,oldusermodelv2score
0,4882710,446322163606528000,446034074208215040,UKU,2020-04-06,7,false,None,628.0,None,None,None
1,4886206,428823284598349824,416134160443867136,UKU,2020-04-08,5,false,None,640.0,None,None,None
2,4886685,443802781886283776,430371457544585216,UKU,2020-04-08,1,false,INTEREST,620.0,None,None,None
3,4880040,443383832304517120,442045235647315968,UKU,2020-04-05,8,false,None,663.0,None,None,None
4,4888705,439022467389227008,439012242435440640,UKU,2020-04-09,2,false,INTEREST,624.0,None,None,None


In [15]:
df0.to_csv(os.path.join(file_path,'data_to_zj.csv'))

# 分析

In [11]:
df0 = load_data_from_pickle(file_path,'data_0414.pkl')
# df0 = load_data_from_pickle(file_path,'data_to_zj.pkl')
df0.head()
df0['newusermodelscorev5'] = df0['newusermodelscorev5'].astype(float)
df0['oldusermodelv1score'] = df0['oldusermodelv1score'].astype(float)
df0['newusermodelscorev6'] = df0['newusermodelscorev6'].astype(float)
df0['oldusermodelv2score'] = df0['oldusermodelv2score'].astype(float)
v5_bin = [ -np.inf,588.0,
 600.0,
 608.0,
 614.0,
 620.0,
 625.0,
 630.0,
 636.0,
 644.5,np.inf]
df0['newusermodelscorev5_bin'] = pd.cut(df0['newusermodelscorev5'],bins = v5_bin,duplicates='drop')


v1_bin = [ -np.inf, 610.59,
 622.0,
 630.97,
 640.0,
 651.58,
 668.71,
 686.34,
 716.59,
 763.23,np.inf]
df0['oldusermodelv1score_bin'] = pd.cut(df0['oldusermodelv1score'],bins = v1_bin,duplicates='drop')

v6_bin = [ -np.inf,  590.0,
 603.0,
 613.0,
 622.0,
 630.0,
 638.0,
 645.0,
 654.0,
 664.0,np.inf]
df0['newusermodelscorev6_bin'] = pd.cut(df0['newusermodelscorev6'],bins = v6_bin,duplicates='drop')

v2_bin = [ -np.inf,  612.0,
 622.0,
 630.0,
 637.0,
 643.0,
 649.0,
 655.0,
 661.0,
 670.0,np.inf]
df0['oldusermodelv2score_bin'] = pd.cut(df0['oldusermodelv2score'],bins = v2_bin,duplicates='drop')
df0.to_excel(os.path.join(file_path,'temp.xlsx'))
print(1)

,case_id,loan_id,customer_id,product,due_date,late_day,return_flag,refund_type,newusermodelscorev5,oldusermodelv1score,newusermodelscorev6,oldusermodelv2score
0,4882710,446322163606528000,446034074208215040,UKU,2020-04-06,7,false,None,628.0,None,None,None
1,4886206,428823284598349824,416134160443867136,UKU,2020-04-08,5,false,None,640.0,None,None,None
2,4886685,443802781886283776,430371457544585216,UKU,2020-04-08,1,false,INTEREST,620.0,None,None,None
3,4880040,443383832304517120,442045235647315968,UKU,2020-04-05,8,false,None,663.0,None,None,None
4,4888705,439022467389227008,439012242435440640,UKU,2020-04-09,2,false,INTEREST,624.0,None,None,None


1
